# Plotly express from Jsonstat

In [1]:
import deetly
import requests
import json
from pyjstat import pyjstat
import pandas as pd
import plotly.express as px
import altair as alt

ModuleNotFoundError: No module named 'pyjstat'

In [20]:
# Create datastory
package = {
    "name": "SSB Labor market example",
    "keywords":["plotly","plotly express","altair","vega","examples"],
}

ds = deetly.story(package)

In [21]:
url = 'https://data.ssb.no/api/v0/no/table/08771'
query = """
{
  "query": [
    {
      "code": "NACE2007",
      "selection": {
        "filter": "all",
        "values": ["*"]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "all",
        "values": ["*"]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "all",
        "values": ["*"]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
"""
res = requests.post(url, json=json.loads(query))
print(res)
dataset = pyjstat.Dataset.read(res.text)
df_stillinger = dataset.write('dataframe')
df_stillinger_id = dataset.write('dataframe',naming='id')
df_stillinger['NACE2007'] = df_stillinger_id['NACE2007']
df_stillinger = df_stillinger[(~df_stillinger['NACE2007'].isin(['00-99','01-96'])) & (df_stillinger['kvartal']>'2010K4')]
df_stillinger = df_stillinger.pivot(index=['NACE2007','næring (SN2007)','kvartal'], 
                                    columns=['statistikkvariabel'], 
                                    values='value').reset_index()



<Response [200]>


In [22]:
df_stillinger.head(1)

statistikkvariabel,NACE2007,næring (SN2007),kvartal,Ledige stillingar,Ledige stillingar (prosent),"Ledige stillingar, endring frå året før","Ledige stillingar, endring i prosentpoeng frå året før"
0,01-03,"Jordbruk, skogbruk og fiske",2011K1,600.0,1.8,300.0,0.8


In [23]:
fig = px.line(df_stillinger, y='Ledige stillingar', x='kvartal', color='NACE2007')
fig.show()

In [24]:
description = """
Ledige stillinger pr næring
"""

ds.plotly(fig, "Ledige stillinger pr næring",description)

## Sysselsatte

In [25]:
url = 'https://data.ssb.no/api/v0/no/table/11154'
query = """
{
  "query": [
    {
      "code": "Kjonn",
      "selection": {
        "filter": "item",
        "values": ["0"]
      }
    },
    {
      "code": "NACE2007",
      "selection": {
        "filter": "all",
        "values": ["*"]
      }
    },
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "all",
        "values": ["*"]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "all",
        "values": ["*"]
      }
    }
  ],
  "response": {
    "format": "json-stat2"
  }
}
"""
res = requests.post(url, json=json.loads(query))
print(res)
dataset = pyjstat.Dataset.read(res.text)
df_sysselsatte = dataset.write('dataframe')
df_sysselsatte_id = dataset.write('dataframe',naming='id')
df_sysselsatte['NACE2007'] = df_sysselsatte_id['NACE2007']
df_sysselsatte = df_sysselsatte[(df_sysselsatte['NACE2007']!='00-99') & (df_sysselsatte['kvartal']>'2010K4')]
df_sysselsatte = df_sysselsatte.pivot(index=['NACE2007','næring (SN2007)','kvartal'], 
                                      columns=['statistikkvariabel'], 
                                      values='value').reset_index()


<Response [200]>


In [26]:
df_sysselsatte.head(1)

statistikkvariabel,NACE2007,næring (SN2007),kvartal,Sysselsatte
0,01-02,Jordbruk og skogbruk,2011K1,45


In [27]:
df_sysselsatte['NACE2007'].unique()

array(['01-02', '03', '05-09', '10-12', '13-15', '16', '17', '18',
       '19-22', '23', '24-25', '26-28', '29-30', '31-33', '35', '36-39',
       '41-43', '45', '46', '47', '49', '50', '51', '52', '53', '55',
       '56', '58-63', '64-66', '68', '69-75', '77-82', '84', '85',
       '86-88', '90-93', '94-99'], dtype=object)

In [28]:
df_stillinger['NACE2007'].unique()

array(['01-03', '05-09', '10-33', '35-39', '41-43', '45-47', '49-53',
       '55-56', '58-63', '64-66', '68', '69-75', '77-82', '84', '85',
       '86', '87', '88', '90-93', '94-96'], dtype=object)

In [29]:
df = pd.merge(df_stillinger, df_sysselsatte, on=['NACE2007','kvartal'], suffixes=['','_sysselsatte'])

In [30]:
df.head(1)

statistikkvariabel,NACE2007,næring (SN2007),kvartal,Ledige stillingar,Ledige stillingar (prosent),"Ledige stillingar, endring frå året før","Ledige stillingar, endring i prosentpoeng frå året før",næring (SN2007)_sysselsatte,Sysselsatte
0,05-09,Bergverksdrift og utvinning,2011K1,1800.0,3.3,1100.0,1.7,Bergverksdrift og utvinning,49


In [31]:
df['NACE2007'].unique()

array(['05-09', '41-43', '58-63', '64-66', '68', '69-75', '77-82', '84',
       '85', '90-93'], dtype=object)

In [32]:
fig = px.scatter(df, x="Ledige stillingar (prosent)", y="Sysselsatte", animation_frame="kvartal", animation_group="næring (SN2007)",
           size="Ledige stillingar", color="næring (SN2007)", hover_name="næring (SN2007)",
           log_x=False, size_max=50)

fig.update_layout(showlegend=False)
fig.show()

In [33]:
description = """
Ledige stillinger og sysselsetting
"""

ds.plotly(fig, "Ledige stillinger og sysselsetting",description)

In [34]:
ds.publish()

Publishing story SSB Labor market example to the examples space...
Published at: https://public.deetly.com/examples/390364409db9bb2dd8016375271ce0c8 

